In [62]:
import pandas as pd
import numpy as np
import random
import math
import sys, os
import json
import time

In [63]:
# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 create-params.py <automation_dir> <seed>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/'
    premade = 0
else:
    # Automation directory
    automation_dir = sys.argv[1]
    premade = sys.argv[2]

premade = {'0': False, 'y': True}[str(premade)]

Usage: python3 create-params2.py <automation_dir> <seed>
Assuming testing directories


In [64]:
def line_value_search(line):
    
    l = []
    for t in line.split():
        try:
            l.append(float(t))
        except ValueError:
            pass
    return l[0]

In [65]:
if not premade:
    var_json = {
        "Events Per Simulation (int)" : 100,
        "Number of Simulations (int)" : 100,
            
        "Parameters" : {}, # [min_value, max_value] uniform distribution or [value, 0] for constant value
        
        "Particles" : { # [particle (name / abbreviation / pdgid), energy (keV), delta (optional)]
            "Example1MeVProton" : ["Proton", 1000], 
            "ExampleProtonAlpha" : ["Proton", 1250, 200, "Alpha", 500, 5] # [particle, energy, delta, particle, energy, delta]
        },
        
        "Pad Threshold" : [86, 0], # [min_value, max_value]
        "Charge Dispersion (int)" : 1,
        "Charge Dispersion H (int)" : 1,
        "Random Seed (int)" : 0,
        "Gas Medium" : "GADGET_P10_800",
        
        "Origin X Bound (cm)" : 2,
        "Origin Y Bound (cm)" : 2,
        "Origin Z Max (cm)" : 40,
        "Origin Z Min (cm)" : 0,
        
        "Approximate Dispersion" : False,
        "High Efficiency Mode" : False,
        "Back to Back Dual Particle Events" : False,
        "Simulation Timer" : False,
    }

    with open(f'{automation_dir}.input/templates/GADGET.sim.par', 'r') as f:
        filedata = f.readlines()

    for line in filedata:
        if line[0] not in ['#', '[']:
            var_json['Parameters'][line.split(" ")[0]] = [line_value_search(line), 0]
            

    # write json file
    with open(f'{automation_dir}ParameterVariation.json', 'w') as f:
        json.dump(var_json, f, indent=4)

In [66]:
def param_range(param):
    if param[1] == 0:
        return [param[0], param[0]]
    else:
        return [param[0], param[1]]

In [67]:
def read_user_input(automation_dir=automation_dir, var_json=var_json):
    # read in user inputs from json file
    with open(f'{automation_dir}ParameterVariation.json', 'r') as f:
        user_json = json.load(f)
    
    if user_json == var_json:
        # user has not modified file yet
        return False
    
    try:
        #read in template parameter csv file
        param_df = pd.read_csv(f'{automation_dir}.input/sim/param.csv')

        # create rows in parameter dataframe
        for i in range(int(user_json['Number of Simulations (int)'])):
            param_df.loc[i] = np.nan

        param_df['Status'] = 0 # unprocessed status
        param_df['N'] = int(user_json['Events Per Simulation (int)']) # number of events per simulation

        if user_json['Random Seed (int)'] != 0:
            param_df['Seed'] = int(user_json['Random Seed (int)'])

        if user_json['Simulation Timer'] == True:
            param_df['Time'] = 0 # initialize timers

        param_df['Xb'] = float(user_json['Origin X Bound (cm)'])
        param_df['Yb'] = float(user_json['Origin Y Bound (cm)'])
        param_df['Zb1'] = float(user_json['Origin Z Min (cm)'])
        param_df['Zb2'] = float(user_json['Origin Z Max (cm)'])

        if user_json['High Efficiency Mode'] == True:
            param_df['Xb'] == 99

        param_df['CD'] = int(user_json['Charge Dispersion (int)'])
        param_df['CDH'] = int(user_json['Charge Dispersion H (int)'])
        if user_json['Approximate Dispersion'] == True:
            param_df['CD'] = param_df['CD'].astype(str) + 'a'

        if user_json['Gas Medium'] != 'GADGET_5IsoAr_800':
            param_df['MediumGas'] = user_json['Gas Medium']

        user_json['Pad Threshold'] = param_range(user_json['Pad Threshold'])
        param_df['Threshold'] = np.random.uniform(user_json['Pad Threshold'][0], user_json['Pad Threshold'][1], size=len(param_df))

        for p in user_json['Parameters'].keys():
            # check if parameter has changed
            if user_json['Parameters'][p] != var_json['Parameters'][p]:
                user_json['Parameters'][p] = param_range(user_json['Parameters'][p])
                param_df[p.split(':')[0]] = np.random.uniform(user_json['Parameters'][p][0], user_json['Parameters'][p][1], size=len(param_df))
                if 'Int_t' in p:
                    param_df[p.split(':')[0]] = param_df[p.split(':')[0]].astype(int)

        # particle types and energies
        particles = list(user_json['Particles'].keys())
        particle_specs = {}
        for p in particles: # make formatting of particle specs consistent
            P0 = user_json['Particles'][p][0]; E0 = float(user_json['Particles'][p][1]) # Main particle
            if len(user_json['Particles'][p]) == 2: # defined p0, no p1
                E0_delta = 0
                P1 = 'Alpha'; E1 = 0; E1_delta = 0 # no second particle, zero energy alpha
            elif len(user_json['Particles'][p]) == 3: # uncertain p0, no p1
                E0_delta = float(user_json['Particles'][p][2])
                P1 = 'Alpha'; E1 = 0; E1_delta = 0
            elif len(user_json['Particles'][p]) == 4: # defined p0, defined p1
                P1 = user_json['Particles'][p][2]; E1 = float(user_json['Particles'][p][3])
                E0_delta = 0; E1_delta = 0
            elif len(user_json['Particles'][p]) == 5: # one defined, one uncertain (worst case)
                # determine if p1 is specified in 2nd or 3rd position
                # ignore possiblity of pdgid for simplicity
                if str(user_json['Particles'][p][2]).lower() in ['p', 'a', 'b', 'g', 'proton', 'alpha', 'beta', 'gamma']: 
                    # defined p0, uncertain p1
                    P1 = user_json['Particles'][p][2]; E1 = float(user_json['Particles'][p][3])
                    E1_delta = float(user_json['Particles'][p][4]); E0_delta = 0
                else: # uncertain p0, defined p1
                    P1 = user_json['Particles'][p][3]; E1 = float(user_json['Particles'][p][4])
                    E1_delta = 0; E0_delta = float(user_json['Particles'][p][2])
            elif len(user_json['Particles'][p]) == 6: # both uncertain
                E0_delta = float(user_json['Particles'][p][2])
                P1 = user_json['Particles'][p][3]; E1 = float(user_json['Particles'][p][4])
                E1_delta = float(user_json['Particles'][p][5])
            particle_specs[p] = [P0, E0, E0_delta, P1, E1, E1_delta] # now consistent format for all particles

        for i,row in param_df.iterrows():
            ptype = particles[i % len(particles)] # particle type
            param_df.loc[i, 'Sim'] = f"Var{ptype}{i//len(particles)}"
            param_df.loc[i, 'P0'] = particle_specs[ptype][0]
            param_df.loc[i, 'E0'] = round(particle_specs[ptype][1] + random.uniform(-1,1)*particle_specs[ptype][2],2)
            param_df.loc[i, 'P1'] = particle_specs[ptype][3]
            param_df.loc[i, 'E1'] = round(particle_specs[ptype][4] + random.uniform(-1,1)*particle_specs[ptype][5],2)

        if user_json['Back to Back Dual Particle Events'] == True:
            param_df['P1'] = '-' + param_df['P1'].astype(str)
        
        param_df.to_csv(f'{automation_dir}parameters.csv', index=False)
        return True
    except:
        print('Error reading user input file')
        return False

In [68]:
if not premade:
    print('Please modify the ParameterVariation.json file to specify the parameters you want to vary.')
    # backup old parameter file if it exists
    if os.path.isfile(f'{automation_dir}parameters.csv'):
        os.system(f'cp {automation_dir}parameters.csv {automation_dir}.input/parameters.bak')

    while True:
        if read_user_input() == True:
            # prompt user for confirmation before running
            print('Parameters updated. Do you want to run simulations? (y/n)')
            while True:
                user_input = input().lower()
                if user_input in ['y', 'yes']: # run simulations
                    print('Closing Variation Script...')
                    os.system(f'rm -f {automation_dir}.input/parameters.bak')
                    if not os.path.isdir(f'{automation_dir}Output'):
                        os.system(f'mkdir {automation_dir}Output')
                    os.system(f'mv -f {automation_dir}ParameterVariation.json {automation_dir}Output/ParameterVariation.json')
                    sys.exit(0) # exit with success
                    break
                elif user_input in ['n', 'no']: # reset environment
                    os.system(f'rm -f {automation_dir}ParameterVariation.json')
                    os.system(f'mv -f {automation_dir}.input/parameters.bak {automation_dir}parameters.csv')
                    sys.exit(1) # exit with error
                    break
                else:
                    print('Please enter y or n')
        else:
            time.sleep(5)
else:
    print('Using Premade ParameterVariation.json file.')
    if read_user_input():
        if not os.path.isdir(f'{automation_dir}Output'):
            os.system(f'mkdir {automation_dir}Output')
        os.system(f'mv -f {automation_dir}ParameterVariation.json {automation_dir}Output/ParameterVariation.json')
        sys.exit(0)
    else:
        sys.exit(1)
    

Please modify the ParameterVariation.json file to specify the parameters you want to vary.
Parameters updated. Do you want to run simulations? (y/n)
Closing Variation Script...


SystemExit: 0

/user/jaros/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
